In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from tqdm import tqdm
from scipy.stats import norm
from scipy.optimize import minimize
from copy import deepcopy
from sklearn.linear_model import LinearRegression
import torch
import torch.nn as nn

In [ ]:
# custom loss function 
def custom_loss(output, target, treatment):
    error = torch.sum(output*treatment,dim=1)
    loss = torch.mean((target - error)**2)
    return loss

# function to calculate the Phi value
def Phi(x,theta,Gamma,treatment,y,device):
    with torch.no_grad():
        theta.eval()
        #device = torch.device('mps')
        a = torch.tensor(x, dtype=torch.float32).to(device)
        output = theta(a).detach().cpu().numpy()
        term1 = np.dot(output,np.array([0,1]))
        term2 = (np.array([0,1]).reshape(1,2)) @ (np.linalg.inv(Gamma)) @ (2*(np.dot(output,treatment) - y)*treatment.reshape(2,1))

    return term1 - term2[0,0]

# function to split the data into S parts
def data_split(S,Hist_feature,Hist_treatment,Hist_label):
    N = Hist_feature.shape[1]
    number_per_split = int(N/S)
    data_index = np.arange(N)
    np.random.shuffle(data_index)
    feature_list = []
    treatment_list = []
    label_list = []

    for i in range(S):
        feature_list.append(Hist_feature[:,data_index[i*number_per_split:(i+1)*number_per_split],:])
        treatment_list.append(Hist_treatment[:,data_index[i*number_per_split:(i+1)*number_per_split],:])
        label_list.append(Hist_label[:,data_index[i*number_per_split:(i+1)*number_per_split]])
    
    return feature_list,treatment_list,label_list

# A simple two-layer fully connected neural network
class TwoLayerFCN(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super(TwoLayerFCN, self).__init__()
        # first layer: input layer to hidden layer
        self.fc1 = nn.Linear(input_dim, hidden_dim)
        # second layer: hidden layer to output layer
        self.fc2 = nn.Linear(hidden_dim, output_dim)
        # activation function
        self.activation = nn.ReLU()

    def forward(self, x):
        # forward pass through the network
        x = self.fc1(x)          
        x = self.activation(x)   
        x = self.fc2(x)          
        return x
    
# Function to train the model
def train_model(S,s,feature_list,treatment_list,label_list,dim_feature,K,device):
    train_data_list = []
    train_label_list = []
    train_treatment_list = []
    for i in range(S):
        if i != s:
            train_data_list.append(feature_list[i])
            train_label_list.append(label_list[i])
            train_treatment_list.append(treatment_list[i])
    train_data_array = np.concatenate(train_data_list, axis=1)
    train_label_array = np.concatenate(train_label_list, axis=1)
    train_treatment_array = np.concatenate(train_treatment_list, axis=1)
   
    
    #estmate the parameter
    model_list = []
    for k in range(K):
        data = train_data_array[k]
        label = train_label_array[k]
        treatment = train_treatment_array[k]
        data = torch.tensor(data, dtype=torch.float32).to(device)
        label = torch.tensor(label, dtype=torch.float32).to(device)
        treatment = torch.tensor(treatment, dtype=torch.float32).to(device)
        model = TwoLayerFCN(dim_feature, 10, 2).to(device)
        optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

        for epoch in range(10):
            batch_size = 64
            index_list = np.random.permutation(data.shape[0])
            data_list = []
            label_list = []
            treatment_list = []
            number_parts = int(data.shape[0]/batch_size)
            for i in range(number_parts):
                if i < number_parts - 1:
                    data_list.append(data[index_list[i*batch_size:(i+1)*batch_size],:])
                    label_list.append(label[index_list[i*batch_size:(i+1)*batch_size]])
                    treatment_list.append(treatment[index_list[i*batch_size:(i+1)*batch_size]])
                else:
                    data_list.append(data[index_list[i*batch_size:],:])
                    label_list.append(label[index_list[i*batch_size:]])
                    treatment_list.append(treatment[index_list[i*batch_size:]])
            for i in range(number_parts):
                model.train()
                output = model(data_list[i])
                #print(output.shape,treatment.shape)
                loss = custom_loss(output, label_list[i], treatment_list[i])
                optimizer.zero_grad()
                loss.backward()
                optimizer.step()
        model_list.append(model)
    
    temp_reshaped = train_treatment_array.reshape(K, train_treatment_array.shape[1], -1, 1)
    outer_product = temp_reshaped @ temp_reshaped.transpose(0, 1, 3, 2) 
    estimated_Gamma = 2 * outer_product.mean(axis=1)  
    
    return model_list, estimated_Gamma


## 1, Boxplot

In [ ]:
dim_feature = 4 # number of features
K = 100 # number of experiments
Z = norm.ppf(0.975) # 95% confidence interval
N = 100 # number of samples per experiment

T = 1000 # number of time steps
S = 2 # number of splits
number_per_split = int(N/S) # number of samples per split
std = 3 # standard deviation of noise


true_ate_array  = np.zeros((T,K))
estimated_phi = np.zeros((T,K,S,number_per_split))
b_k = np.zeros((T,K))
device = torch.device('cpu')
for t in tqdm(range(T)):
    constant_linear = np.random.uniform(-0.3,0.5,(K,dim_feature))

    # linear true function
    cofficent_linear = np.random.uniform(-0.3,0.5,(K,dim_feature))

    true_ate = np.sum(cofficent_linear[:,:]*0.5, axis=1)
    true_ate_array[t,:] = true_ate


    #generate data
    Hist_feature = np.random.uniform(0,1,(K,N,dim_feature))

    Hist_treatment = np.ones((K, N, 2))
    Hist_treatment[:, int(N/2):, 1] = 0

    temp_feature = np.zeros((K, N, dim_feature + 1))
    temp_feature[:,:,0] = Hist_feature[:,:,0]
    temp_feature[:,:,1] = Hist_treatment[:,:,1]
    temp_feature[:,:,2:] = Hist_feature[:,:,1:]
    
    for k in range(K):
        a = temp_feature[k].T@temp_feature[k]
        b = np.linalg.inv(a + np.eye(dim_feature + 1)*10**(-5))
        b_k[t,k] = N*b[1,1]



    # constant linear true function
    Hist_label = np.zeros((K, N))
    for i in range(N):
        Hist_label[:,i] = np.sum(Hist_feature[:,i,:]*constant_linear, axis=1) + np.sum(Hist_feature[:,i,:]*cofficent_linear, axis=1)*Hist_treatment[:,i,1] + np.random.normal(0,std,K)

    #split data
    feature_list,treatment_list,label_list = data_split(S,Hist_feature,Hist_treatment,Hist_label)

    #train model
    for s in range(S):
        model_list,estimated_Gamma = train_model(S,s,feature_list,treatment_list,label_list,dim_feature,K,device)
        for k in range(K):
            feature_ate = feature_list[s][k]
            treatment_ate = treatment_list[s][k]
            label_ate = label_list[s][k]
            theta = model_list[k]
            Gamma = estimated_Gamma[k]
            for i in range(feature_ate.shape[0]):
                phi_value = Phi(feature_ate[i],theta,Gamma,treatment_ate[i],label_ate[i],device)
                estimated_phi[t,k,s,i] = phi_value


In [ ]:
cost = np.zeros((3,T))
accuracy = np.zeros((2,1000))
recall = np.zeros((2,1000))
FPR = np.zeros((2,1000))
precision = np.zeros((2,1000))

for t in range(T):
    hat_ATE = np.mean(estimated_phi[t],axis=(1,2))
    estimated_variance1 = np.mean((estimated_phi[t] - np.tile(hat_ATE.reshape(K,1,1),(S,number_per_split)))**2,axis=(1,2))
    estimated_variance = np.mean((estimated_phi[t] - np.tile(hat_ATE.reshape(K,1,1),(S,number_per_split)))**2)
    true_tao = true_ate_array[t]
    optimal_cost = np.sum(true_tao[np.argwhere(true_tao>0)])
    if optimal_cost == 0:
        continue
    cost[0,t] = np.sum(true_tao[np.argwhere(hat_ATE>(Z*np.sqrt(estimated_variance1))/np.sqrt(N))])/optimal_cost
    decision1 = np.argwhere(hat_ATE>(Z*np.sqrt(estimated_variance1))/np.sqrt(N))

    #DRT
    anchor_tau = np.mean(hat_ATE)
    beta = estimated_variance/(np.mean((hat_ATE - anchor_tau)**2) - estimated_variance/N) + Z*np.sqrt(N*estimated_variance)/anchor_tau
    beta = max(0,beta)
    theta = N/(N+beta)
    hat_ATE_shrunken = theta*hat_ATE + (1-theta)*anchor_tau
    cost[1,t] = np.sum(true_tao[np.argwhere(hat_ATE_shrunken>(theta*Z*np.sqrt(estimated_variance1))/np.sqrt(N))])/optimal_cost
    decision2 = np.argwhere(hat_ATE_shrunken>(theta*Z*np.sqrt(estimated_variance1))/np.sqrt(N))

    #DRT-P
    estimated_variance_p = np.mean(estimated_variance1/b_k[t])
    theta_list = np.zeros(K)
    for k in range(K):
        beta_p = estimated_variance_p*b_k[t,k]/(np.mean((hat_ATE - anchor_tau)**2) - (estimated_variance_p*np.mean(b_k[t]))/N) + Z*b_k[t,k]*np.sqrt(N*estimated_variance_p)/anchor_tau
        beta_p = max(0,beta_p)
        theta_list[k] = N/(N+beta_p)
    hat_ATE_shrunken_p = theta_list*hat_ATE + (1-theta_list)*anchor_tau

    cost[2,t] = np.sum(true_tao[np.argwhere(hat_ATE_shrunken_p>(theta_list*Z*np.sqrt(estimated_variance1))/np.sqrt(N))])/optimal_cost


    for k in range(K):
        if (true_tao[k] < 0 and k not in decision1) or (true_tao[k] > 0 and k in decision1):
            accuracy[0,t] += 1
        if (true_tao[k] < 0 and k not in decision2) or (true_tao[k] > 0 and k in decision2):
            accuracy[1,t] += 1
        if true_tao[k] > 0 and k in decision1:
            recall[0,t] += 1
        if true_tao[k] > 0 and k in decision2:
            recall[1,t] += 1
        if true_tao[k] < 0 and k in decision1:
            FPR[0,t] += 1
        if true_tao[k] <0 and k in decision2:
            FPR[1,t] += 1
    if recall[0,t] +FPR[0,t] == 0:
        precision[0,t] = 1
    else:
        precision[0,t] = recall[0,t]/(recall[0,t] +FPR[0,t] )
    if recall[1,t] +FPR[1,t] == 0:
        precision[1,t] = 1
    else:
        precision[1,t] = recall[1,t]/(recall[1,t] +FPR[1,t] )
    accuracy[:,t] = accuracy[:,t]/K
    recall[:,t] = recall[:,t]/(len(np.argwhere(true_tao>0)))
    FPR[:,t] = FPR[:,t]/(len(np.argwhere(true_tao<0)))

In [ ]:
fig, ax = plt.subplots(figsize = (8,6))
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)
data = [cost[1,:] - cost[0,:],accuracy[1,:] - accuracy[0,:],recall[1,:] - recall[0,:],FPR[0,:] - FPR[1,:],precision[1,:] - precision[0,:]]

bp = plt.boxplot(data,showfliers=False,showmeans=True,patch_artist=True)

colors = [ '#9DB4CE','#EDA1A4','#FCB462','#7BC4C5','#893E81']
colors1 = ['#A3A5A6' ,'#A3A5A6','#FFE8CE','#D9EEEE','#DCA5C3']

for patch, color in zip(bp['boxes'], colors):
    patch.set_facecolor(color)
    patch.set_edgecolor(color)
    
for whisker, color in zip(bp['whiskers'], [colors[i // 2] for i in range(len(bp['whiskers']))]):
    whisker.set_color(color)

for cap, color in zip(bp['caps'], [colors[i // 2] for i in range(len(bp['caps']))]):
    cap.set_color(color)

for median, color in zip(bp['medians'], colors1):
    median.set_color(color)

for flier, color in zip(bp['fliers'], [colors[i // 2] for i in range(len(bp['fliers']))]):
    flier.set_markerfacecolor(color)
    flier.set_markeredgecolor(color)

plt.ylim(-1.1,1)
plt.yticks(fontsize=10)
plt.grid(axis='y', linestyle='--', alpha=0.6)

plt.xticks([1, 2,3,4,5], ['OR','Accuracy','Recall','Specificity','Precision'],fontsize=15)


plt.savefig('boxplot_basic_dml.png',dpi = 300,bbox_inches = 'tight')
plt.show()

## 2, Cost with varianc of noise term

In [ ]:
dim_feature = 4
K = 100 
#K = 5
Z = norm.ppf(0.975)
N = 100

T = 1000
S = 2
number_per_split = int(N/S)
noise_list = [1,2,3,4,5] # standard deviation of noise

In [ ]:
true_ate_array  = np.zeros((T,K))
estimated_phi = np.zeros((len(noise_list),T,K,S,number_per_split))
b_k = np.zeros((T,K))
device = torch.device('cpu')
for t in tqdm(range(T)):
    constant_linear = np.random.uniform(-0.3,0.5,(K,dim_feature))

    # linear true function
    cofficent_linear = np.random.uniform(-0.3,0.5,(K,dim_feature))

    true_ate = np.sum(cofficent_linear[:,:]*0.5, axis=1)
    true_ate_array[t,:] = true_ate


    #generate data
    Hist_feature = np.random.uniform(0,1,(K,N,dim_feature))

    Hist_treatment = np.ones((K, N, 2))
    Hist_treatment[:, int(N/2):, 1] = 0

    temp_feature = np.zeros((K, N, dim_feature + 1))
    temp_feature[:,:,0] = Hist_feature[:,:,0]
    temp_feature[:,:,1] = Hist_treatment[:,:,1]
    temp_feature[:,:,2:] = Hist_feature[:,:,1:]
    
    for k in range(K):
        a = temp_feature[k].T@temp_feature[k]
        b = np.linalg.inv(a + np.eye(dim_feature + 1)*10**(-5))
        b_k[t,k] = N*b[1,1]


    for id,std in enumerate(noise_list):
    # constant linear true function
        Hist_label = np.zeros((K, N))
        for i in range(N):
            Hist_label[:,i] = np.sum(Hist_feature[:,i,:]*constant_linear, axis=1) + np.sum(Hist_feature[:,i,:]*cofficent_linear, axis=1)*Hist_treatment[:,i,1] + np.random.normal(0,std,K)

        #split data
        feature_list,treatment_list,label_list = data_split(S,Hist_feature,Hist_treatment,Hist_label)

        #train model
        for s in range(S):
            model_list,estimated_Gamma = train_model(S,s,feature_list,treatment_list,label_list,dim_feature,K,device)
            for k in range(K):
                feature_ate = feature_list[s][k]
                treatment_ate = treatment_list[s][k]
                label_ate = label_list[s][k]
                theta = model_list[k]
                Gamma = estimated_Gamma[k]
                for i in range(feature_ate.shape[0]):
                    phi_value = Phi(feature_ate[i],theta,Gamma,treatment_ate[i],label_ate[i],device)
                    estimated_phi[id,t,k,s,i] = phi_value

In [ ]:
cost = np.zeros((len(noise_list),3,T))
accuracy = np.zeros((2,1000))
recall = np.zeros((2,1000))
FPR = np.zeros((2,1000))
precision = np.zeros((2,1000))
for i in range(len(noise_list)):
    for t in range(T):
        hat_ATE = np.mean(estimated_phi[i,t],axis=(1,2))
        estimated_variance1 = np.mean((estimated_phi[i,t] - np.tile(hat_ATE.reshape(K,1,1),(S,number_per_split)))**2,axis=(1,2))
        estimated_variance = np.mean((estimated_phi[i,t] - np.tile(hat_ATE.reshape(K,1,1),(S,number_per_split)))**2)
        true_tao = true_ate_array[t]
        optimal_cost = np.sum(true_tao[np.argwhere(true_tao>0)])
        if optimal_cost == 0:
            continue
        cost[i,0,t] = np.sum(true_tao[np.argwhere(hat_ATE>(Z*np.sqrt(estimated_variance1))/np.sqrt(N))])/optimal_cost
        decision1 = np.argwhere(hat_ATE>(Z*np.sqrt(estimated_variance1))/np.sqrt(N))

        #DRT
        anchor_tau = np.mean(hat_ATE)
        beta = estimated_variance/(np.mean((hat_ATE - anchor_tau)**2) - estimated_variance/N) + Z*np.sqrt(N*estimated_variance)/anchor_tau
        beta = max(0,beta)
        theta = N/(N+beta)
        hat_ATE_shrunken = theta*hat_ATE + (1-theta)*anchor_tau
        cost[i,1,t] = np.sum(true_tao[np.argwhere(hat_ATE_shrunken>(theta*Z*np.sqrt(estimated_variance1))/np.sqrt(N))])/optimal_cost
        decision2 = np.argwhere(hat_ATE_shrunken>(theta*Z*np.sqrt(estimated_variance1))/np.sqrt(N))

        #DRT-P
        estimated_variance_p = np.mean(estimated_variance1/b_k[t])
        theta_list = np.zeros(K)
        for k in range(K):
            beta_p = estimated_variance_p*b_k[t,k]/(np.mean((hat_ATE - anchor_tau)**2) - (estimated_variance_p*np.mean(b_k[t]))/N) + Z*b_k[t,k]*np.sqrt(N*estimated_variance_p)/anchor_tau
            beta_p = max(0,beta_p)
            theta_list[k] = N/(N+beta_p)
        hat_ATE_shrunken_p = theta_list*hat_ATE + (1-theta_list)*anchor_tau

        cost[i,2,t] = np.sum(true_tao[np.argwhere(hat_ATE_shrunken_p>(theta_list*Z*np.sqrt(estimated_variance1))/np.sqrt(N))])/optimal_cost